In [5]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from scipy.stats import linregress

In [33]:
def read_excecution_time(df):
    df = df.rename(columns={'Column1': 'N', 'Column2': 'CORES', 'Column3': 'TIEMPO_EJECUCION', 'Source.Name': 'EJECUCION'})
    df['CORES'] = df['CORES'].str.replace(':','')
    df['CORES'] = df['CORES'].astype(int)
    df['TIEMPO_EJECUCION'] = pd.to_numeric(df['TIEMPO_EJECUCION'])  
    return df 

# Lectura del archivo que contiene los tiempos de la ejecución de la multiplicación de matrices usando el algoritmo filas x columnas (maquina 1)
FilasxColumnasM1 = pd.read_excel('FilasxColumnasL2.xlsx', sheet_name='cad2')
FilasxColumnasM1 = read_excecution_time(FilasxColumnasM1)

# Lectura del archivo que contiene los tiempos de la ejecución de la multiplicación de matrices usando el algoritmo filas x columnas (maquina 2)
FilasxColumnasM2 = pd.read_excel('FilasxColumnasL2M2.xlsx', sheet_name='cad2 (2)')
FilasxColumnasM2 = read_excecution_time(FilasxColumnasM2)

# Lectura del archivo que contiene los tiempos de la ejecución de la multiplicación de matrices usando el algoritmo filas x columnas (maquina 3)
FilasxColumnasM3 = pd.read_excel('FilasxColumnasL2M3.xlsx', sheet_name='cad2 (2)')
FilasxColumnasM3 = read_excecution_time(FilasxColumnasM3)

# Lectura del archivo que contiene los tiempos de la ejecución de la multiplicación de matrices usando el algoritmo filas x filas (maquina 1)
FilasxFilasM1 = pd.read_excel('FilasxFilasL2.xlsx', sheet_name='cad6 (2)')
FilasxFilasM1 = read_excecution_time(FilasxFilasM1)

# Lectura del archivo que contiene los tiempos de la ejecución de la multiplicación de matrices usando el algoritmo filas x filas (maquina 2)
FilasxFilasM2 = pd.read_excel('FilasxFilasL2M2.xlsx', sheet_name='cad6 (2)')
FilasxFilasM2 = read_excecution_time(FilasxFilasM2)

# Lectura del archivo que contiene los tiempos de la ejecución de la multiplicación de matrices usando el algoritmo filas x filas (maquina 3)
FilasxFilasM3 = pd.read_excel('FilasxFilasL2M3.xlsx', sheet_name='cad6 (2)')
FilasxFilasM3 = read_excecution_time(FilasxFilasM3)

A continuación, calculamos los tiempos promedio por cada combinación de los parámetros ```N``` y ```CORES```. Esto lo haremos agrupando por cada combinación.

In [34]:
FilasxColumnasM1_promedio = FilasxColumnasM1.groupby(['N', 'CORES'], as_index=False)['TIEMPO_EJECUCION'].mean()
FilasxColumnasM2_promedio = FilasxColumnasM2.groupby(['N', 'CORES'], as_index=False)['TIEMPO_EJECUCION'].mean()
FilasxColumnasM3_promedio = FilasxColumnasM3.groupby(['N', 'CORES'], as_index=False)['TIEMPO_EJECUCION'].mean()
FilasxFilasM1_promedio = FilasxFilasM1.groupby(['N', 'CORES'], as_index=False)['TIEMPO_EJECUCION'].mean()
FilasxFilasM2_promedio = FilasxFilasM2.groupby(['N', 'CORES'], as_index=False)['TIEMPO_EJECUCION'].mean()
FilasxFilasM3_promedio = FilasxFilasM3.groupby(['N', 'CORES'], as_index=False)['TIEMPO_EJECUCION'].mean()

Ajustamos los campos ```CORES``` y ```TIEMPO_EJECUCION```, de modo que la cantidad de cores quede como un valor mayor a 1 y el tiempo de ejecución esté expresado en segundos.

In [35]:
def ajustar_numero_cores(cantidad_cores):
    return cantidad_cores + 1

def milisegundos_a_segundos(milisegundos):
    segundos = milisegundos / 1000
    return segundos

FilasxColumnasM1_promedio['CORES'] = FilasxColumnasM1_promedio['CORES'].apply(ajustar_numero_cores)
FilasxColumnasM1_promedio['TIEMPO_EJECUCION'] = FilasxColumnasM1_promedio['TIEMPO_EJECUCION'].apply(milisegundos_a_segundos)
FilasxColumnasM2_promedio['CORES'] = FilasxColumnasM2_promedio['CORES'].apply(ajustar_numero_cores)
FilasxColumnasM2_promedio['TIEMPO_EJECUCION'] = FilasxColumnasM2_promedio['TIEMPO_EJECUCION'].apply(milisegundos_a_segundos)
FilasxColumnasM3_promedio['CORES'] = FilasxColumnasM3_promedio['CORES'].apply(ajustar_numero_cores)
FilasxColumnasM3_promedio['TIEMPO_EJECUCION'] = FilasxColumnasM3_promedio['TIEMPO_EJECUCION'].apply(milisegundos_a_segundos)
FilasxFilasM1_promedio['CORES'] = FilasxFilasM1_promedio['CORES'].apply(ajustar_numero_cores)
FilasxFilasM1_promedio['TIEMPO_EJECUCION'] = FilasxFilasM1_promedio['TIEMPO_EJECUCION'].apply(milisegundos_a_segundos)
FilasxFilasM2_promedio['CORES'] = FilasxFilasM2_promedio['CORES'].apply(ajustar_numero_cores)
FilasxFilasM2_promedio['TIEMPO_EJECUCION'] = FilasxFilasM2_promedio['TIEMPO_EJECUCION'].apply(milisegundos_a_segundos)
FilasxFilasM3_promedio['CORES'] = FilasxFilasM3_promedio['CORES'].apply(ajustar_numero_cores)
FilasxFilasM3_promedio['TIEMPO_EJECUCION'] = FilasxFilasM3_promedio['TIEMPO_EJECUCION'].apply(milisegundos_a_segundos)

Con lo anterior, podemos calcular las medidas de eficiencia escogidas previamente para el experimento, es decir, el **Speed Up** y la **eficiencia**. Que recordemos, se calculan de la siguiente manera:

$$
Speedup = \frac{T_{1}}{T_{n}},
$$

donde $T_{1}$ es el tiempo de ejecución con un solo procesador y $T_{n}$ es el tiempo de ejecución utilizando n procesadores; y

$$
Eficiencia = \frac{Speedup}{\# CORES}
$$

In [36]:
def calculate_speedup_and_efficiency(df):
    secuencial_time = df.groupby('N')['TIEMPO_EJECUCION'].min()

    speed_up_values = []
    efficiency_values = []

    for n_value in df['N'].unique():
        secuencial_time = df[df['N'] == n_value]['TIEMPO_EJECUCION'].min()
        for index, row in df.iterrows():
            if row['N'] == n_value:
                speed_up = secuencial_time / row['TIEMPO_EJECUCION']
                efficiency = speed_up / row['CORES']
                speed_up_values.append(speed_up)
                efficiency_values.append(efficiency)
    
    df['SPEED_UP'] = speed_up_values
    df['EFICIENCIA'] = efficiency_values
    return df

FilasxColumnasM1_promedio = calculate_speedup_and_efficiency(FilasxColumnasM1_promedio)
FilasxColumnasM2_promedio = calculate_speedup_and_efficiency(FilasxColumnasM2_promedio)
FilasxColumnasM3_promedio = calculate_speedup_and_efficiency(FilasxColumnasM3_promedio)
FilasxFilasM1_promedio = calculate_speedup_and_efficiency(FilasxFilasM1_promedio)
FilasxFilasM2_promedio = calculate_speedup_and_efficiency(FilasxFilasM2_promedio)
FilasxFilasM3_promedio = calculate_speedup_and_efficiency(FilasxFilasM3_promedio)

In [37]:
def graph_speedup(df):
    fig = go.Figure()
    for n_value in df['N'].unique():
        fig.add_trace(go.Scatter(x=df[df['N'] == n_value]['CORES'], y=df[df['N'] == n_value]['SPEED_UP'], 
                                 mode='lines+markers', name=f'N={n_value}'))
    
    fig.update_layout(title='Gráfico de Speed Up para diferentes valores de N',
                      xaxis_title='Cantidad de cores',
                      yaxis_title='Speed Up')
    
    fig.show()

In [38]:
from plotly.subplots import make_subplots

In [39]:
colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']

In [68]:
comparacion_prueba1 = pd.merge(left=FilasxColumnasM1_promedio, right=FilasxFilasM1_promedio, how='inner', on=['N', 'CORES'])
comparacion_prueba2 = pd.merge(left=FilasxColumnasM2_promedio, right=FilasxFilasM2_promedio, how='inner', on=['N', 'CORES'])
comparacion_prueba3 = pd.merge(left=FilasxColumnasM3_promedio, right=FilasxFilasM2_promedio, how='inner', on=['N', 'CORES'])

fig = make_subplots(rows=3, cols=2)
for i, n_value in enumerate(comparacion_prueba1['N'].unique()):
    color = colors[i % len(colors)]  # Seleccionar un color de la paleta
    # Gráficas sobre la maquina 1
    fig.add_trace(go.Scatter(x=comparacion_prueba1[comparacion_prueba1['N'] == n_value]['CORES'], 
                             y=comparacion_prueba1[comparacion_prueba1['N'] == n_value]['SPEED_UP_x'], mode='lines+markers', name=f'N={n_value}', line=dict(color=color), showlegend=True, legendgroup=f"N={n_value}"), row=1, col=1)
    fig.add_trace(go.Scatter(x=comparacion_prueba1[comparacion_prueba1['N'] == n_value]['CORES'], 
                             y=comparacion_prueba1[comparacion_prueba1['N'] == n_value]['SPEED_UP_y'], mode='lines+markers', name=f'N={n_value}', line=dict(color=color), showlegend=True, legendgroup=f"N={n_value}"), row=1, col=2)
    # Gráficas sobre la maquina 2
    fig.add_trace(go.Scatter(x=comparacion_prueba2[comparacion_prueba2['N'] == n_value]['CORES'], 
                             y=comparacion_prueba2[comparacion_prueba2['N'] == n_value]['SPEED_UP_x'], mode='lines+markers', name=f'N={n_value}', line=dict(color=color), showlegend=True, legendgroup=f"N={n_value}"), row=2, col=1)
    fig.add_trace(go.Scatter(x=comparacion_prueba2[comparacion_prueba2['N'] == n_value]['CORES'], 
                             y=comparacion_prueba2[comparacion_prueba2['N'] == n_value]['SPEED_UP_y'], mode='lines+markers', name=f'N={n_value}', line=dict(color=color), showlegend=True, legendgroup=f"N={n_value}"), row=2, col=2)
    # Gráficas sobre la maquina 3
    fig.add_trace(go.Scatter(x=comparacion_prueba3[comparacion_prueba3['N'] == n_value]['CORES'], 
                             y=comparacion_prueba3[comparacion_prueba3['N'] == n_value]['SPEED_UP_x'], mode='lines+markers', name=f'N={n_value}', line=dict(color=color), showlegend=True, legendgroup=f"N={n_value}"), row=3, col=1)
    fig.add_trace(go.Scatter(x=comparacion_prueba3[comparacion_prueba3['N'] == n_value]['CORES'], 
                             y=comparacion_prueba3[comparacion_prueba3['N'] == n_value]['SPEED_UP_y'], mode='lines+markers', name=f'N={n_value}', line=dict(color=color), showlegend=True, legendgroup=f"N={n_value}"), row=3, col=2)

# Actualizar el diseño del gráfico
fig.update_xaxes(title_text="Cores", row=1, col=1)
fig.update_yaxes(title_text="Speed up", row=1, col=1)

fig.update_xaxes(title_text="Cores", row=1, col=2)
fig.update_yaxes(title_text="Speed up", row=1, col=2)

fig.update_xaxes(title_text="Cores", row=2, col=1)
fig.update_yaxes(title_text="Speed up", row=2, col=1)

fig.update_xaxes(title_text="Cores", row=2, col=2)
fig.update_yaxes(title_text="Speed up", row=2, col=2)

fig.update_xaxes(title_text="Cores", row=3, col=1)
fig.update_yaxes(title_text="Speed up", row=3, col=1)

fig.update_xaxes(title_text="Cores", row=3, col=2)
fig.update_yaxes(title_text="Speed up", row=3, col=2)

fig.update_layout(annotations=[
    dict(text="Filas x Columnas", x=0.18, y=1.1, xref="paper", yref="paper", showarrow=False, font=dict(size=20)),
    dict(text="Filas x Filas", x=0.825, y=1.1, xref="paper", yref="paper", showarrow=False, font=dict(size=20))
])

fig.update_layout(height=800, width=1000)
# Mostrar el gráfico
fig.show()

A continuación graficaremos el tiempo de ejecución

In [67]:
fig = make_subplots(rows=3, cols=2)
for i, n_value in enumerate(comparacion_prueba2['N'].unique()):
    color = colors[i % len(colors)]  # Seleccionar un color de la paleta
    # Gráficas sobre la maquina 1
    fig.add_trace(go.Scatter(x=comparacion_prueba1[comparacion_prueba1['N'] == n_value]['CORES'], 
                             y=comparacion_prueba1[comparacion_prueba1['N'] == n_value]['TIEMPO_EJECUCION_x'], mode='lines+markers', name=f'N={n_value}', line=dict(color=color), showlegend=True, legendgroup=f"N={n_value}"), row=1, col=1)
    fig.add_trace(go.Scatter(x=comparacion_prueba1[comparacion_prueba1['N'] == n_value]['CORES'], 
                             y=comparacion_prueba1[comparacion_prueba1['N'] == n_value]['TIEMPO_EJECUCION_y'], mode='lines+markers', name=f'N={n_value}', line=dict(color=color), showlegend=True, legendgroup=f"N={n_value}"), row=1, col=2)
    # Gráficas sobre la maquina 2
    fig.add_trace(go.Scatter(x=comparacion_prueba2[comparacion_prueba2['N'] == n_value]['CORES'], 
                             y=comparacion_prueba2[comparacion_prueba2['N'] == n_value]['TIEMPO_EJECUCION_x'], mode='lines+markers', name=f'N={n_value}', line=dict(color=color), showlegend=True, legendgroup=f"N={n_value}"), row=2, col=1)
    fig.add_trace(go.Scatter(x=comparacion_prueba2[comparacion_prueba2['N'] == n_value]['CORES'], 
                             y=comparacion_prueba2[comparacion_prueba2['N'] == n_value]['TIEMPO_EJECUCION_y'], mode='lines+markers', name=f'N={n_value}', line=dict(color=color), showlegend=True, legendgroup=f"N={n_value}"), row=2, col=2)
    # Gráficas sobre la maquina 3
    fig.add_trace(go.Scatter(x=comparacion_prueba3[comparacion_prueba3['N'] == n_value]['CORES'], 
                             y=comparacion_prueba3[comparacion_prueba3['N'] == n_value]['TIEMPO_EJECUCION_x'], mode='lines+markers', name=f'N={n_value}', line=dict(color=color), showlegend=True, legendgroup=f"N={n_value}"), row=3, col=1)
    fig.add_trace(go.Scatter(x=comparacion_prueba3[comparacion_prueba3['N'] == n_value]['CORES'], 
                             y=comparacion_prueba3[comparacion_prueba3['N'] == n_value]['TIEMPO_EJECUCION_y'], mode='lines+markers', name=f'N={n_value}', line=dict(color=color), showlegend=True, legendgroup=f"N={n_value}"), row=3, col=2)

# Actualizar el diseño del gráfico
fig.update_xaxes(title_text="Cores", row=1, col=1)
fig.update_yaxes(title_text="Tiempo de ejecución", row=1, col=1)

fig.update_xaxes(title_text="Cores", row=1, col=2)
fig.update_yaxes(title_text="Tiempo de ejecución", row=1, col=2)

fig.update_xaxes(title_text="Cores", row=2, col=1)
fig.update_yaxes(title_text="Tiempo de ejecución", row=2, col=1)

fig.update_xaxes(title_text="Cores", row=2, col=2)
fig.update_yaxes(title_text="Tiempo de ejecución", row=2, col=2)

fig.update_xaxes(title_text="Cores", row=3, col=1)
fig.update_yaxes(title_text="Tiempo de ejecución", row=3, col=1)

fig.update_xaxes(title_text="Cores", row=3, col=2)
fig.update_yaxes(title_text="Tiempo de ejecución", row=3, col=2)

fig.update_layout(annotations=[
    dict(text="Filas x Columnas", x=0.18, y=1.1, xref="paper", yref="paper", showarrow=False, font=dict(size=20)),
    dict(text="Filas x Filas", x=0.825, y=1.1, xref="paper", yref="paper", showarrow=False, font=dict(size=20))
])

fig.update_layout(height=800, width=1000)
# Mostrar el gráfico
fig.show()

Finalmente, graficaremos la eficiencia

In [69]:
axis_range = [0, 1.1]

fig = make_subplots(rows=3, cols=2)
for i, n_value in enumerate(comparacion_prueba1['N'].unique()):
    color = colors[i % len(colors)]  # Seleccionar un color de la paleta
    # Gráficas sobre la maquina 1
    fig.add_trace(go.Scatter(x=comparacion_prueba1[comparacion_prueba1['N'] == n_value]['CORES'], 
                             y=comparacion_prueba1[comparacion_prueba1['N'] == n_value]['EFICIENCIA_x'], mode='lines+markers', name=f'N={n_value}', 
                             line=dict(color=color), showlegend=True, legendgroup=f"N={n_value}"), row=1, col=1)
    fig.update_yaxes(range=axis_range, row=1, col=1)
    fig.add_trace(go.Scatter(x=comparacion_prueba1[comparacion_prueba1['N'] == n_value]['CORES'], 
                             y=comparacion_prueba1[comparacion_prueba1['N'] == n_value]['EFICIENCIA_y'], mode='lines+markers', name=f'N={n_value}', 
                             line=dict(color=color), showlegend=True, legendgroup=f"N={n_value}"), row=1, col=2)
    fig.update_yaxes(range=axis_range, row=1, col=2)
    # Gráficas sobre la maquina 2
    fig.add_trace(go.Scatter(x=comparacion_prueba2[comparacion_prueba2['N'] == n_value]['CORES'], 
                             y=comparacion_prueba2[comparacion_prueba2['N'] == n_value]['EFICIENCIA_x'], mode='lines+markers', name=f'N={n_value}', 
                             line=dict(color=color), showlegend=True, legendgroup=f"N={n_value}"), row=2, col=1)
    fig.update_yaxes(range=axis_range, row=2, col=1)
    fig.add_trace(go.Scatter(x=comparacion_prueba2[comparacion_prueba2['N'] == n_value]['CORES'], 
                             y=comparacion_prueba2[comparacion_prueba2['N'] == n_value]['EFICIENCIA_y'], mode='lines+markers', name=f'N={n_value}', 
                             line=dict(color=color), showlegend=True, legendgroup=f"N={n_value}"), row=2, col=2)
    fig.update_yaxes(range=axis_range, row=2, col=2)
    # Gráficas sobre la maquina 3
    fig.add_trace(go.Scatter(x=comparacion_prueba3[comparacion_prueba3['N'] == n_value]['CORES'], 
                             y=comparacion_prueba3[comparacion_prueba3['N'] == n_value]['EFICIENCIA_x'], mode='lines+markers', name=f'N={n_value}', 
                             line=dict(color=color), showlegend=True, legendgroup=f"N={n_value}"), row=3, col=1)
    fig.update_yaxes(range=axis_range, row=3, col=1)
    fig.add_trace(go.Scatter(x=comparacion_prueba3[comparacion_prueba3['N'] == n_value]['CORES'], 
                             y=comparacion_prueba3[comparacion_prueba3['N'] == n_value]['EFICIENCIA_y'], mode='lines+markers', name=f'N={n_value}', 
                             line=dict(color=color), showlegend=True, legendgroup=f"N={n_value}"), row=3, col=2)
    fig.update_yaxes(range=axis_range, row=3, col=2)

# Actualizar el diseño del gráfico
fig.update_xaxes(title_text="Cores", row=1, col=1)
fig.update_yaxes(title_text="Eficiencia", row=1, col=1)

fig.update_xaxes(title_text="Cores", row=1, col=2)
fig.update_yaxes(title_text="Eficiencia", row=1, col=2)

fig.update_xaxes(title_text="Cores", row=2, col=1)
fig.update_yaxes(title_text="Eficiencia", row=2, col=1)

fig.update_xaxes(title_text="Cores", row=2, col=2)
fig.update_yaxes(title_text="Eficiencia", row=2, col=2)

fig.update_xaxes(title_text="Cores", row=3, col=1)
fig.update_yaxes(title_text="Eficiencia", row=3, col=1)

fig.update_xaxes(title_text="Cores", row=3, col=2)
fig.update_yaxes(title_text="Eficiencia", row=3, col=2)

fig.update_layout(annotations=[
    dict(text="Filas x Columnas", x=0.18, y=1.1, xref="paper", yref="paper", showarrow=False, font=dict(size=20)),
    dict(text="Filas x Filas", x=0.825, y=1.1, xref="paper", yref="paper", showarrow=False, font=dict(size=20))
])

fig.update_layout(height=800, width=1000)
# Mostrar el gráfico
fig.show()

## Comparación momento 1 vs momento 2

In [51]:
comparacion_prueba1 = pd.merge(left=FilasxColumnasM1_promedio, right=FilasxFilasM1_promedio, how='inner', on=['N', 'CORES'])
comparacion_prueba2 = pd.merge(left=FilasxColumnasM2_promedio, right=FilasxFilasM2_promedio, how='inner', on=['N', 'CORES'])
comparacion_prueba3 = pd.merge(left=FilasxColumnasM3_promedio, right=FilasxFilasM3_promedio, how='inner', on=['N', 'CORES'])

In [71]:
def graph_linear_regresion(df, field='EFICIENCIA'):
    # Aplicación de la regresión lineal sobre las eficiencias calculadas para los dos algoritmos
    slope, intercept, r_value, p_value, std_err = linregress(comparacion_prueba1[f'{field}_x'].values, comparacion_prueba1[f'{field}_y'].values)
    modelo_lineal = np.poly1d([slope, intercept])
    predicciones = modelo_lineal(comparacion_prueba1[f'{field}_x'].values)
    r_cuadrado = r_value ** 2
    
    fig = go.Figure()
    for n_value in df['N'].unique():
        fig.add_trace(go.Scatter(x=df[df['N'] == n_value][f'{field}_x'].values, y=df[df['N'] == n_value][f'{field}_y'].values, 
                                 mode='markers', name=f'N={n_value}'))
    
    fig.add_trace(go.Scatter(x=comparacion_prueba1[f'{field}_x'].values, y=predicciones, mode='lines', name='Identidad', line=dict(color='black', dash='dash')))
    
    fig.update_layout(
        title='Gráfico de eficiencia para diferentes valores de N con identidad'
    )
    print('R2: ', r_cuadrado)
    print('P valor: ', p_value)
    fig.show()

In [73]:
# EFICIENCIA, TIEMPO_EJECUCION, SPEED_UP
graph_linear_regresion(comparacion_prueba1, "EFICIENCIA")

R2:  0.9176514279099574
P valor:  3.3446346589037073e-22


In [74]:
graph_linear_regresion(comparacion_prueba2, "EFICIENCIA")

R2:  0.9176514279099574
P valor:  3.3446346589037073e-22


In [75]:
graph_linear_regresion(comparacion_prueba3, "EFICIENCIA")

R2:  0.9176514279099574
P valor:  3.3446346589037073e-22
